<a href="https://colab.research.google.com/github/Eldar1999/image-processing-lessons/blob/main/lab123.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Копирование изображений из репозитория

In [2]:
!git clone https://github.com/Eldar1999/image-processing-lessons

fatal: destination path 'image-processing-lessons' already exists and is not an empty directory.


# Lab 1

In [3]:
import cv2
import numpy as np
import os
import glob

from google.colab.patches import cv2_imshow

1. Определение размеров шахматной доски
2. Определение критериев для завершения поиска углов доски





In [4]:

CHECKERBOARD = (10,14)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

3. Создание вектора для хранения векторов трехмерных точек для каждого изображения шахматной доски
4. Создание вектора для хранения векторов 2D точек для каждого изображения шахматной доски

In [5]:
objpoints = []
imgpoints = []



5. Определение мировых координат для 3D точек


In [6]:
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

6. Извлечение пути каждого изображения, хранящегося в данном каталоге

In [13]:
images = glob.glob('image-processing-lessons/Калибровка/*.JPG')
print(images)

['image-processing-lessons/Калибровка/DSC02701.JPG', 'image-processing-lessons/Калибровка/DSC02712.JPG', 'image-processing-lessons/Калибровка/DSC02711.JPG', 'image-processing-lessons/Калибровка/DSC02700.JPG', 'image-processing-lessons/Калибровка/DSC02709.JPG', 'image-processing-lessons/Калибровка/DSC02703.JPG', 'image-processing-lessons/Калибровка/DSC02715.JPG', 'image-processing-lessons/Калибровка/DSC02707.JPG', 'image-processing-lessons/Калибровка/DSC02704.JPG', 'image-processing-lessons/Калибровка/DSC02717.JPG', 'image-processing-lessons/Калибровка/DSC02714.JPG', 'image-processing-lessons/Калибровка/DSC02710.JPG', 'image-processing-lessons/Калибровка/DSC02698.JPG', 'image-processing-lessons/Калибровка/DSC02699.JPG', 'image-processing-lessons/Калибровка/DSC02708.JPG', 'image-processing-lessons/Калибровка/DSC02705.JPG', 'image-processing-lessons/Калибровка/DSC02718.JPG', 'image-processing-lessons/Калибровка/DSC02706.JPG', 'image-processing-lessons/Калибровка/DSC02713.JPG', 'image-proc

7. Нахождение шахматных досок на изображениях

In [14]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Найти углы шахматной доски
    # Если на изображении найдено нужное количество углов, тогда ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    """
    Если желаемый номер угла обнаружен,
    уточняем координаты пикселей и отображаем
    их на изображениях шахматной доски
    """
    if ret == True:
        objpoints.append(objp)
        # уточнение координат пикселей для заданных 2d точек.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
      
        imgpoints.append(corners2)

        # Нарисовать и отобразить углы
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)

8. Нахождение матриц внутренней калибровки и векторов внешней калибровки камеры

In [15]:
h,w = img.shape[:2]

"""
Выполнение калибровки камеры с помощью
Передача значения известных трехмерных точек (объектов)
и соответствующие пиксельные координаты
обнаруженные углы (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

Camera matrix : 
[[6.05304390e+03 0.00000000e+00 2.54106629e+03]
 [0.00000000e+00 6.03790774e+03 1.30293219e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 
[[-1.75617464e-01  3.62500845e+00  3.27014373e-03  3.28190318e-03
  -2.11790229e+01]]
rvecs : 
(array([[ 0.14253079],
       [ 0.49760035],
       [-1.46534763]]), array([[ 0.4719614 ],
       [-0.48683858],
       [ 1.51214199]]), array([[ 0.21265419],
       [ 0.32576342],
       [-1.54874568]]), array([[ 0.25418409],
       [ 0.40757497],
       [-1.52602739]]), array([[ 0.12259148],
       [-0.48170536],
       [ 1.49137047]]), array([[ 0.03741832],
       [ 0.56015617],
       [-1.42965708]]), array([[-0.18425612],
       [-0.32930909],
       [ 1.54722524]]), array([[ 0.26227936],
       [-0.3047019 ],
       [ 1.55465791]]), array([[ 0.06934424],
       [ 0.50863308],
       [-1.45178828]]), array([[ 0.13273162],
       [ 0.34120534],
       [-1.53727919]]), array([[ 0.23040169],
       [-0.69636237],
       [ 1

9. Вывод матрицы внутренней калибровки камеры:

In [16]:
print(mtx)

[[6.05304390e+03 0.00000000e+00 2.54106629e+03]
 [0.00000000e+00 6.03790774e+03 1.30293219e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


10. Векторы смещения и поворота (внешняя калибровка камеры)

In [18]:
print(tvecs, rvecs, sep="\n-------\n")

(array([[-7.42993952],
       [ 4.30122647],
       [44.71096096]]), array([[ 5.75805601],
       [-1.36248787],
       [27.79380441]]), array([[-9.65158301],
       [ 4.90336566],
       [43.26733668]]), array([[-8.17607566],
       [ 4.74698351],
       [44.09389422]]), array([[ 5.59241517],
       [-4.20438452],
       [37.54949232]]), array([[-7.74135456],
       [ 4.12483033],
       [46.97901238]]), array([[ 6.96558865],
       [-4.17689362],
       [30.6233645 ]]), array([[ 6.57502485],
       [-1.93346832],
       [42.0671126 ]]), array([[-7.58813982],
       [ 4.31168139],
       [47.78933034]]), array([[-9.0163729 ],
       [ 3.25328823],
       [34.87532001]]), array([[ 6.76323311],
       [-3.55547877],
       [29.68805234]]), array([[-8.35918279],
       [ 5.23375611],
       [40.6373608 ]]), array([[ 7.17294622],
       [-3.28344254],
       [37.43479602]]), array([[-6.31275515],
       [ 4.87902847],
       [43.66594731]]), array([[-6.88115125],
       [ 6.28070836],
   